# Jerarquia de indices
pg 128

In [1]:
import pandas as pd
import numpy as np

Pandas proporciona los objetos `Panel` y `Panel4D` nativamente para tres y cuatro dimensiones pero la manera mas aceptada es usar indices jerarquizados.

### MultiIndex

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
index = pd.MultiIndex.from_tuples(index)

In [4]:
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [5]:
pop = pop.reindex(index)

In [6]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiindex como dimensión extra

In [7]:
# Convertir un multiindex en DataFrame convencional.
pop_df = pop.unstack()

In [8]:
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [10]:
# Operación inversa.
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Creación de multiindex

Lo mas comun es pasar una lista con dos o más arrays.

In [11]:
df = pd.DataFrame(np.random.rand(4 , 2),
                  index=([['a', 'a', 'b', 'b'], [1, 2, 1, 2]]),
                  columns=['data1', 'data2'])

In [12]:
df

data1     data2
a 1  0.113079  0.780607
  2  0.651715  0.297023
b 1  0.593454  0.196741
  2  0.330609  0.116468

### Constructores explicitos

In [13]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [14]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [15]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
# En el libro viene con labels. Debe estar deprecado y ahora es codes
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### Nombres en los indices

In [20]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multi para columnas

In [22]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      51.0  35.3  52.0  37.7  41.0  36.1
     2      36.0  37.4  25.0  37.6  42.0  36.4
2014 1      22.0  36.5  43.0  36.8  52.0  38.6
     2      38.0  37.3  59.0  36.9  31.0  37.4

### Slicing en multiindex

Funciona igual que le normal.

In [21]:
pop['California', 2000]

33871648

In [23]:
health_data['Guido', 'HR']

year  visit
2013  1        52.0
      2        25.0
2014  1        43.0
      2        59.0
Name: (Guido, HR), dtype: float64

In [29]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      51.0  35.3
     2      36.0  37.4

Si queremos hacer un slicing más complejo se aconseja usar `IndexSlice`

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]
# Todos los años, donde sea visita 1. Todos los sujetos, el HR

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,51.0,52.0,41.0
2014,1,22.0,43.0,52.0


### Reordenando multi-indices

Trabajar con datos efectivamente es realizar operaciones preservando la info en el dataset pero ordenandola y seleccioandola con varios propositos de computación.

### Indices ordenados y desordenados

Muchas de las operaciones de slicing en multi-index no funcionan si los indices no están ordenados

In [35]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.433018
      2      0.003271
c     1      0.083807
      2      0.105119
b     1      0.818180
      2      0.757171
dtype: float64

In [36]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Pandas tiene métodos para este caso.

In [37]:
data = data.sort_index()
data

char  int
a     1      0.433018
      2      0.003271
b     1      0.818180
      2      0.757171
c     1      0.083807
      2      0.105119
dtype: float64

In [38]:
data['a':'b']

char  int
a     1      0.433018
      2      0.003271
b     1      0.818180
      2      0.757171
dtype: float64

### Stack y unstack

In [39]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [40]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [41]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Setear indice y resetear

Convertir el indice en columnas

In [42]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Normalente los datos reales lucen así. Uno de los métodos más útiles es `set_index`

In [43]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Agregado de datos en multi-indice

Se puede indicar el nivel para realizar el agrupado en ese indice

In [44]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      51.0  35.3  52.0  37.7  41.0  36.1
     2      36.0  37.4  25.0  37.6  42.0  36.4
2014 1      22.0  36.5  43.0  36.8  52.0  38.6
     2      38.0  37.3  59.0  36.9  31.0  37.4

In [45]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     43.5  36.35  38.5  37.65  41.5  36.25
2014     30.0  36.90  51.0  36.85  41.5  38.00

In [46]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,41.166667,36.75
2014,40.833333,37.25
